In [ ]:
from src.data_util import load_preprocess_examples, normalize_aorta
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from src.data_util import load_augmented_example

In [ ]:
def analyse_measurements(X, y, labels):
    pca = PCA(n_components=2, svd_solver="full")
    pca.fit(X)
    Xt = pca.transform(X)

    pca = PCA(n_components=2, svd_solver="full")
    pca.fit(y)
    yt = pca.transform(y)

    fig, axs = plt.subplots(3, 2, figsize=(16, 32))

    axs[0, 0].plot(np.sum(X, axis=1) / 1024)
    axs[0, 0].set_title(f"EIT -- {labels[0]}")
    axs[0, 0].grid()
    axs[0, 0].set_ylabel("sum EIT signals")
    axs[0, 0].set_ylim([-1.5, 1.5])

    axs[1, 0].hist(X.flatten(), bins=100, range=[-3, 3])
    axs[1, 0].set_ylabel("count")
    axs[1, 0].set_ylabel("amplitude")

    axs[2, 0].scatter(Xt[:, 0], Xt[:, 1], s=0.25, alpha=1)
    axs[2, 0].set_xlabel("PCA Axis 0")
    axs[2, 0].set_ylabel("PCA Axis 1")
    axs[2, 0].grid()

    axs[0, 1].plot((normalize_aorta(np.hstack(y), invert=True)))
    axs[0, 1].grid()
    axs[0, 1].set_title(f"Aorta Pressure -- {labels[0]}")
    axs[0, 1].set_ylabel("pressure in mmHG")
    # axs[1,1].set_ylim([0, 1.2])

    axs[1, 1].hist(normalize_aorta(y, invert=True).flatten(), bins=100)
    axs[1, 1].set_ylabel("count")
    axs[1, 1].set_ylabel("amplitude")

    axs[2, 1].scatter(yt[:, 0], yt[:, 1], s=0.25, alpha=1)
    axs[2, 1].set_xlabel("PCA Axis 0")
    axs[2, 1].set_ylabel("PCA Axis 1")
    axs[2, 1].grid()

    # fig.savefig(f'./quality_checks/PCA_{labels[0]}.png')


def plot_aorta(y, labels):
    y10 = np.array_split(y, 10)

    fig, axs = plt.subplots(5, 2, figsize=(16, 40))

    for n in range(2):
        for m in range(5):
            axs[m, n].plot(
                normalize_aorta(y10[(n - 1) * 5 + m].T, invert=True), "k-", alpha=0.1
            )
            axs[m, n].grid()
            # axs[n].set_title(f'Aorta Pressure -- {labels[0]}')
            axs[m, n].set_ylabel("pressure mmHg")

    fig.suptitle(f"Aorta Pressure -- {labels[0]}")

    # fig.savefig(f'./quality_checks/aorta_{labels[0]}.png')

In [ ]:
train_pig = ["P_{0:02d}_PulHyp".format(i) for i in range(1, 11)]
print(train_pig)

In [ ]:
for pig in train_pig:
    X, y, _ = load_augmented_example("/data/PulHyp_augmented_SNR20/", [pig])
    X = np.reshape(X, (X.shape[0] * X.shape[1], X.shape[2]))

    scaler_X = StandardScaler()
    scaler_X.fit(X)
    X = scaler_X.transform(X)

    analyse_measurements(X, y, pig)
    plot_aorta(y, pig)